In [1]:
# !unzip /content/src.zip -d /content/
# !unzip /content/data.zip -d /content/

**Install Dependencies**

In [2]:
!pip install -qq -U \
  langchain \
  langgraph \
  chromadb \
  faiss-cpu \
  transformers \
  accelerate \
  bitsandbytes \
  sentence-transformers \
  pypdf \
  tavily-python


In [3]:
import langchain
import langgraph
import chromadb
import transformers
import torch

print("LangChain:", langchain.__version__)
# print("LangGraph:", langgraph.__version__)
print("Chroma:", chromadb.__version__)
print("Transformers:", transformers.__version__)
print("CUDA available:", torch.cuda.is_available())

LangChain: 1.2.0
Chroma: 1.4.0
Transformers: 4.57.3
CUDA available: True


In [4]:
from src.llm.model import load_llm
import torch

tokenizer, model = load_llm()

prompt = "Explain what retrieval augmented generation is in one sentence."
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

with torch.no_grad():
    outputs = model.generate(**inputs, max_new_tokens=50)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Explain what retrieval augmented generation is in one sentence. Retrieval augmented generation is a method that combines the strengths of retrieval models and generation models to produce more accurate and contextually relevant responses.

Retrieval augmented generation (RAG) is a cutting-edge approach that


In [5]:
!pip install -U langchain-community

In [6]:
!pip install -U langchain-text-splitters

In [1]:
from src.ingestion.ingest import load_papers, chunk_documents
from src.ingestion.vectorstore import build_vectorstore

docs = load_papers()
print(f"Loaded pages: {len(docs)}")

chunks = chunk_documents(docs)
print(f"Created chunks: {len(chunks)}")

vectordb = build_vectorstore(chunks)

Loaded pages: 119
Created chunks: 973


/content/src/ingestion/vectorstore.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/content/src/ingestion/vectorstore.py:1

In [8]:
from src.ingestion.ingest import load_papers, chunk_documents
from src.ingestion.vectorstore import build_vectorstore
from src.rag.baseline import BaselineRAG

# Load vector store
docs = load_papers()
chunks = chunk_documents(docs)
vectordb = build_vectorstore(chunks)

# Initialize baseline RAG
rag = BaselineRAG(vectordb)

query = "How does knowledge distillation improve small object detection performance in YOLO-based models?"

answer = rag.answer(query)
print(answer)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


You are a research assistant.

Use the provided context to answer the question.
If the answer is not explicitly stated, try your best to infer it.

Context:
Source: paper_2.pdf
[4] G. Chen, W. Choi, X. Yu, T. Han, and M. Chandraker, “Learning efﬁ-
cient object detection models with knowledge distillation,” inProc. Adv.
Neural Inf. Process. Syst., 2017, pp. 742–751.
[5] Z. Xing, X. Chen, and F. Pang, “DD-YOLO: An object detection method
combining knowledge distillation and differentiable architecture search,”
IET Comput. Vis., vol. 16, pp. 418–430, 2022.
[6] Z. Li et al., “A compression pipeline for one-stage object detection
model,” J. Real-Time Image Process., vol. 18, pp. 1949–1962, 2021.
[7] G. Hinton, O. Vinyals, and J. Dean, “Distilling the knowledge in a neural
network,” 2015,arXiv:1503.02531.

Source: paper_2.pdf
[4] G. Chen, W. Choi, X. Yu, T. Han, and M. Chandraker, “Learning efﬁ-
cient object detection models with knowledge distillation,” inProc. Adv.
Neural Inf. Process. Sys

In [9]:
query = ["Which distillation loss is most effective for small object detection according to the papers?",
"Do feature-level or response-level distillation methods perform better across datasets?",
"What teacher-student architectures consistently outperform baselines?"]
for q in query:
    answer = rag.answer(q)
    print(answer)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


You are a research assistant.

Use the provided context to answer the question.
If the answer is not explicitly stated, try your best to infer it.

Context:
Source: paper_10.pdf
loss as follows:
L= Loriginal + Lfocal + Lglobal (13)
where Loriginal is the original loss for detectors.
The distillation loss is calculated just on feature maps,
which can be obtained from the neck of the detectors. So it
can be easily applied to different detectors.
4. Experiments
4.1. Dataset
We evaluate our knowledge distillation method on
COCO dataset [21], which contains 80 object classes. We
use the 120k train images for training and 5k val images
for testing for all the experiments. The performances of
different detectors are evaluated in Average Precision and
Average Recall.
Method mAP AP S APM APL
RetinaNet-Res101(T) 38.9 21.0 42.8 52.4
RetinaNet-Res50(S) 37.4 20.6 40.7 49.7

Source: paper_10.pdf
loss as follows:
L= Loriginal + Lfocal + Lglobal (13)
where Loriginal is the original loss for detectors.

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


You are a research assistant.

Use the provided context to answer the question.
If the answer is not explicitly stated, try your best to infer it.

Context:
Source: paper_6.pdf
Distillation w/o FM-NMSVOCOCO61.1 (-3.1) 61.8 (-2.1)
Distillation w/o Obj-scalingVOCOCO65.8 (+1.2) 64.9 (+1.0)
Distillation full VOCOCO66.9 (+2.7) 66.1 (2.1)
Table 3: Comparison of performance for distillation with
different strategies on Pascal VOC 2007. The results are
shown for two teacher network and for two set of labeled
training data (Pascal VOC and combination of Pascal VOC
and COCO).
feature map merging on the base architecture. The results
for different layer combinations are shown in Table 1. It can
be observed that the accuracy increases as the feature maps
from more layers are merged together. Another important
inference that we can draw from these results is that merg-

Source: paper_6.pdf
Distillation w/o FM-NMSVOCOCO61.1 (-3.1) 61.8 (-2.1)
Distillation w/o Obj-scalingVOCOCO65.8 (+1.2) 64.9 (+1.0)

In [10]:
from src.ingestion.ingest import load_papers, chunk_documents
from src.ingestion.vectorstore import build_vectorstore
from src.agents.relevance_grader import RelevanceGrader

docs = load_papers()
chunks = chunk_documents(docs)
vectordb = build_vectorstore(chunks)

query = "How does feature-level distillation improve small object detection?"
retrieved = vectordb.similarity_search(query, k=3)

grader = RelevanceGrader()

for doc in retrieved:
    result = grader.grade(query, doc)
    print(result)
    print(doc.page_content[:200])
    print("----")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


{'relevant': 'yes', 'reason': 'The document chunk discusses the use of feature-level distillation in object detection, which is specifically mentioned in the question.'}
to the web version of this article.)
the feature-based distillation, which utilizes the semantic information
of feature. Unlike the response-based distillation only in the output
layer, feature-based 
----


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


{'relevant': 'yes', 'reason': 'The document chunk discusses the use of feature-level distillation in object detection, which is specifically mentioned in the question.'}
to the web version of this article.)
the feature-based distillation, which utilizes the semantic information
of feature. Unlike the response-based distillation only in the output
layer, feature-based 
----
{'relevant': 'yes', 'reason': 'The document chunk discusses the use of feature-level distillation in object detection, which is specifically mentioned in the question.'}
to the web version of this article.)
the feature-based distillation, which utilizes the semantic information
of feature. Unlike the response-based distillation only in the output
layer, feature-based 
----


In [7]:
!zip /content/data -r /content/

  adding: content/ (stored 0%)
  adding: content/.config/ (stored 0%)
  adding: content/.config/config_sentinel (stored 0%)
  adding: content/.config/active_config (stored 0%)
  adding: content/.config/default_configs.db (deflated 98%)
  adding: content/.config/configurations/ (stored 0%)
  adding: content/.config/configurations/config_default (deflated 15%)
  adding: content/.config/gce (stored 0%)
  adding: content/.config/.last_survey_prompt.yaml (stored 0%)
  adding: content/.config/.last_opt_in_prompt.yaml (stored 0%)
  adding: content/.config/logs/ (stored 0%)
  adding: content/.config/logs/2025.12.09/ (stored 0%)
  adding: content/.config/logs/2025.12.09/14.41.18.717681.log (deflated 58%)
  adding: content/.config/logs/2025.12.09/14.41.42.675750.log (deflated 57%)
  adding: content/.config/logs/2025.12.09/14.40.47.605300.log (deflated 92%)
  adding: content/.config/logs/2025.12.09/14.41.33.792924.log (deflated 58%)
  adding: content/.config/logs/2025.12.09/14.41.27.893750.log (d

In [4]:
from src.graph.workflow import build_graph

graph = build_graph(vectordb)

initial_state = {
    "query": "How does knowledge distillation help small object detection?",
    "documents": [],
    "relevant_docs": [],
    "retry_count": 0,
}

final_state = graph.invoke(initial_state)

print("Final Query:", final_state["query"])
print("Relevant chunks:", len(final_state["relevant_docs"]))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Final Query: "What is the role of knowledge distillation in enhancing small object detection performance?"
Relevant chunks: 6


In [6]:
from src.agents.answer_generator import AnswerGenerator
from src.ingestion.ingest import load_papers, chunk_documents
from src.ingestion.vectorstore import build_vectorstore

docs = load_papers()
chunks = chunk_documents(docs)
vectordb = build_vectorstore(chunks)
query = "How does knowledge distillation help small object detection?"

retrieved = vectordb.similarity_search(query, k=5)

generator = AnswerGenerator()

answer = generator.generate(
    question=query,
    documents=retrieved
)

print(answer)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Knowledge distillation helps small object detection by transferring useful structure and semantic information from a larger, more complex "teacher" model to a smaller, simpler "student" model. This is done to improve the accuracy of the student model, which in this context is a detection model. The information is transferred through a combination of structure loss (Lsize) and offset loss (Loff), along with the original detection loss (Lk). The overall training objective for the detection model includes these losses, with tunable parameters to balance their contributions.


In [5]:
# from src.graph.workflow import build_graph
# from src.ingestion.ingest import load_papers, chunk_documents
# from src.ingestion.vectorstore import build_vectorstore

# docs = load_papers()
# print(f"Loaded pages: {len(docs)}")

# chunks = chunk_documents(docs)
# print(f"Created chunks: {len(chunks)}")

# vectordb = build_vectorstore(chunks)
# graph = build_graph(vectordb)
# initial_state = {
#     "query": "What is Closed-loop unified knowledge distillation for dense object detection?",
#     "retry_count": 0
# }

# final_state = graph.invoke(initial_state)

# print("Final Query:", final_state["query"])
# print("Retries:", final_state["retry_count"])
# print("Relevant Chunks:", len(final_state["relevant_docs"]))
# print("\nANSWER:\n")
# print(final_state["answer"])

Loaded pages: 119
Created chunks: 973


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Final Query: What is Closed-loop unified knowledge distillation for dense object detection?
Retries: 0
Relevant Chunks: 6

ANSWER:



KeyError: 'answer'